In [ ]:
import pandas as pd

x1 = pd.ExcelFile('re_test.xlsx')
df = x1.parse('Sheet1')  # Replace 'Sheet1' with the name of your sheet if it's different

df = df.rename(columns={'HouseNumber': 'raw address'})
print(df.columns)
addresses = df['raw address'].tolist()

print(len(addresses))
print(addresses)

Index(['StartedDateTime', 'AssignedDateTime', 'ExternalFacilityId',
       'CompletedBy', 'Vendor', 'BuildingAccessible', 'raw address',
       'CompassPoint', 'StreetName', 'Artery', 'zipcode', 'ZipCode'],
      dtype='object')
15456
['27-09 Murray St New York NY 11354 US', '116 W 74th st New York  US 10023', '346 E 84 Street ', '130 East 64th Street New York New York 10065 United States', '301  E 90th St New York NY 10128 US', '200-38 39 Avenue  New York  New York  11361', '150-03 BAYSIDE AVE  New York  New York ', '150-58 BAYSIDE AVE  New York  New York  11354', '113 East 64th Street New York New York 10065 United States', '157 W 75th ST New York City New York 10023 US', '144-59 28th Ave New York NY 11354 US', '131 E 24th Street  AKA 50 Lexington Avenue New York NY 10010 US', '125 East 64th Street New York New York 10065 United States', '153-10 75th Ave New York NY 11367 US', '1322 2nd Ave', '147-09 35th Ave Queens NY 11354 US', '144-35 Sanford Ave  Flushing  New York ', '153-15 75t

In [ ]:
import re

def process_data(address):
    if not isinstance(address, str):
      return ['', '', '', '', '']

    house_number_re = re.compile(r'^[\d-]+')
    compass_point_re = re.compile(r'(?<= )([NESW]|North|East|South|West)(?= )', re.IGNORECASE)
    street_name_re = re.compile(r'(?:(?<=[NESW] )|(?<= ))[\w\s-]+(?= (St|Street|Rd|Road|Ave|Avenue|Blvd|Boulevard))', re.IGNORECASE)
    artery_re = re.compile(r'(?<= )(St|Street|Rd|Road|Ave|Avenue|Blvd|Boulevard)', re.IGNORECASE)
    zip_code_re = re.compile(r'\b\d{5}\b')

    compass_dict = {
    's': 'South',
    'n': 'North',
    'e': 'East',
    'w': 'West',
    }

    artery_dict = {
    'st': 'Street',
    'av': 'Avenue',
    'ave': 'Avenue',
    'rd': 'Road',
    'blvd':'Boulevard'
    }

    print(address)
    address = re.sub(r'United States', '', address, flags=re.IGNORECASE)
    address = re.sub(r'New York', '', address, 1, flags=re.IGNORECASE) if address.lower().count('new york') > 1 else address

    house_number = house_number_re.search(address)
    house_number = house_number.group() if house_number else ''

    compass_point = compass_point_re.search(address)
    if compass_point:

      compass_point = compass_point.group()
      oc = compass_point
      compass_point = compass_dict.get(compass_point.lower(),compass_point)
    else:
      compass_point= ''

    street_name = street_name_re.search(address)
    if street_name:
        street_name = street_name.group().strip()
        # If the street name starts with a compass point, remove it
        if compass_point and street_name.startswith(oc):
            street_name = street_name[len(oc)+1:].strip() # +1 added to handle the space after compass point.
    else:
        street_name = ''

    artery = artery_re.search(address)
    if artery:
        artery = artery.group()
        artery = artery_dict.get(artery.lower(), artery)
    else:
        artery = ''

    zip_code = zip_code_re.search(address)
    zip_code = zip_code.group() if zip_code else ''

    return [house_number, compass_point, street_name, artery, zip_code]


formatted_addresses = [process_data(address) for address in addresses]
# print(formatted_addresses)




Streaming output truncated to the last 5000 lines.
374 Eighth Ave New York NY 10001 US
372 8th Avenue New York NY 10001 US
301 W 22nd St New York NY
145-84 6 Ave 
12-13  151 Pl
4162 Bowne Street Queens New York 11355
468 W 47 NYC New York  10019 US
23-14a Corporal Kennedy St New York NY 11360 US
256 W 21st St New York NY 10011 US
47-57 Utopia Pkwy Queens  New York  11358 US
4 E Second St New York NY 10003 US
1619 Broadway Manhattan  New York  10019
110 Seaman Ave New York  NY 10034
853 9 Avenue 
421 E 76th St New York NY 10021 US
35-38 160th St New York NY 11358 US
404 E 80th St
58-15 Hollis CRT BLVD New York NY 11364 US
4 East 3rd  New York NY 10003 US
240 E 76th St New York NY 10021 US
1530 Broadway  NYC New York  10019 US
1066 2nd Ave  New York NY 10022 US
14731  69th Rd Queens  New York 11367
49-04 43rd Ave New York NY 11377 US
1717 Clintonville Street Queens New York 11357
77 Payson Avenue  New York  NY 10034
670 9th Ave NYC New York  10019 US
149-50 Sanford Ave Queens  New York  

In [ ]:
df_new = pd.DataFrame(formatted_addresses, columns=['HouseNumber', 'CompassPoint', 'StreetName', 'Artery', 'ZipCode'])

# concatenate the original DataFrame with the new one
df_final = pd.concat([df, df_new], axis=1)

# write the final DataFrame to a new Excel file
df_final.to_excel('real data_formatted.xlsx', index=False)